In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import linear_model
sns.set_style("whitegrid")

In [ ]:
%matplotlib inline

Before we get started on learning about Linear Regression let's load in our ATUS data so we've got a starting point.

In [ ]:
summary = pd.read_csv("atussum_2013.dat")
summary.info()

Here is a handy function to give us back a `dataFrame` containing ages with mean minutes for any given category.

In [ ]:
def activity_by_age(df, activity_code, subsample=True):
    activity_col = "t{}".format(activity_code)
    df = df[['TUFINLWGT', 'TEAGE', activity_col]]
    df = df.rename(columns={"TUFINLWGT": "weight", "TEAGE": "age", activity_col: "minutes"})
    if subsample:
        df = df[df.age % 5 == 0]
    df['weighted_minutes'] = df.weight * df.minutes
    df = df.groupby("age").sum()
    df['mean_minutes'] = df.weighted_minutes / df.weight
    df = df[['mean_minutes']]
    return df

Lets get a `dataFrame` from our summary containing "Medical Care" data.

The code for Medical Care is `080401`.

Also do a scatter plot with the `medical_care.index` as the x, and the `medical_care.mean_minutes` as the y.

<!---
medical_care = activity_by_age(summary, "080401", subsample=False)
plt.scatter(medical_care.index, medical_care.mean_minutes)
--->

## Linear Regression

Linear regression is a way of modeling a relationship between two features in a way that given 2 variables, 1 known, and 1 unknown we can give a close approximation of what our unknown value may equate to.

Or as wikipedia says:
> In linear regression, data are modeled using linear predictor functions, and unknown model parameters are estimated from the data 

Without getting too in depth with how to calculate the predictor function (our line) our goal is going to be to get the minimum (closest to the mean) of our data along an axis.  If our prediction is too aggressive (steep) we will not get an accurate function.  If our prediction is too conservator (shallow) we will also not get an accurate function.  You ultimately want your line to visibly bisect your distribution of data.

The function below will apply a regression function across a dataframe and square the differences in each cell from the mean.  Finally it will divide the sum of differences by 2 times the amount of differences we find.

In [ ]:
def linear_least_squares(df, fn):
    values = df.index.map(fn)
    diffs = df.mean_minutes - values
    diffs_squared = diffs ** 2
    return diffs_squared.sum() / (2 * len(diffs)) 

In order to use our `linear_least_squares` function we need to pass it a function.  This function will act as our "line prediction" in which we are attempting to correctly bisect our data.

This is the code to plot a line with a slope of `1/4`:
>`lambda x: 0 + 0.25 * x`

Should I explain linear slope?  Remember - I used to have to calculate roof slope for a living.

Pass the `medical_care` frame to the `linear_least_squares` along with the function above and print your findings.

Also create a scatter plot of your data as well as your regression line prediction.

<!---
fn1 = lambda x: 0 + 0.25 * x
print(linear_least_squares(medical_care, fn1))
plt.scatter(medical_care.index, medical_care.mean_minutes)
xmin, xmax = plt.xlim()
xs = np.array([xmin, xmax])
plt.plot(xs, fn1(xs))
--->

Try the same but this time with the following linear function:

>`lambda x: 0 + 0.15 * x`

Overlay your original line with an alpha of `0.3`

<!---
fn2 = lambda x: 0 + 0.15 * x
print(linear_least_squares(medical_care, fn2))
plt.scatter(medical_care.index, medical_care.mean_minutes)
xmin, xmax = plt.xlim()
xs = np.array([xmin, xmax])
plt.plot(xs, fn1(xs), alpha=0.3)
plt.plot(xs, fn2(xs))
--->

Finally lets try with the following function:

>`lambda x: 0 + 0.1 * x`

Overlay previous lines with an alpha of `0.3`

<!---
fn3 = lambda x: 0 + 0.1 * x
print(linear_least_squares(medical_care, fn3))
plt.scatter(medical_care.index, medical_care.mean_minutes)
xmin, xmax = plt.xlim()
xs = np.linspace(xmin, xmax, 100)
plt.plot(xs, fn1(xs), alpha=0.3)
plt.plot(xs, fn2(xs), alpha=0.3)
plt.plot(xs, fn3(xs))
--->

Neat! But this isn't really a realistic way to figure out a regression line. Better yet we should use a library to predict our line for us given our data!  We can do this by creating a `LinearRegression()` model and calling it's `fit` method with our data.

<!---
regression = linear_model.LinearRegression()
transposed_index = np.array(medical_care.index.values).reshape((-1, 1))
regression.fit(transposed_index, medical_care.mean_minutes.values)
--->

Now that we have fit our prediction model to our data lets do cool stuff to it.

In [ ]:
regression.predict(60)

In [ ]:
print(regression.coef_, regression.intercept_)

In [ ]:
regression.score(np.array(medical_care.index.values).reshape((-1, 1)), 
               medical_care.mean_minutes.values)

In [ ]:
fn = lambda x: regression.predict(x)[0]
print(linear_least_squares(medical_care, fn))
plt.scatter(medical_care.index, medical_care.mean_minutes)
xmin, xmax = plt.xlim()
xs = np.linspace(xmin, xmax, 100)
plt.plot(xs, [fn(x) for x in xs])

In [ ]:
linear_least_squares(medical_care, lambda x: regression.predict(x)[0])